In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, concat, col, lit, from_json, window, sum, to_json, struct, count
from pyspark.sql.functions import slice as F_slice
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, TimestampType, MapType, IntegerType, ArrayType
from time import sleep

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Batch_process")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Structure of the expected data
schema = StructType([
     StructField('type', StringType(), True),
     StructField('symbol_id', StringType(), True),
     StructField('sequence', IntegerType(), True),
     StructField('time_exchange', TimestampType(), True),
     StructField('time_coinapi', TimestampType(), True),
     StructField('uuid', StringType(), True),
     StructField('price', DoubleType(), True),
     StructField('size', DoubleType(), True),
     StructField('taker_side', StringType(), True),
     ])

# Spark dataframe, reading the trade topic
df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka1:9093") \
        .option("subscribe", "trade") \
        .option("startingOffsets", "latest") \
        .option("failOnDataLoss", "false") \
        .load()

# Cast kafka value to string, cast the JSON formatted string to the structure we want, then unnest (data.* => *)
trades = df.select(col('value').cast('string')).select(from_json(col('value'), schema).alias('data')).select('data.*')

# Select columns that we are interested in
reduced_trades = trades.select(['symbol_id', 'time_coinapi', 'size', 'taker_side', 'uuid'])

# Watermark the data to get clean results, group by minute, symbol_id and taker side
# Then take the sum of the size and count all the unique trades
# Make the results more readable
trades_aggregated = reduced_trades.withWatermark('time_coinapi', '1 minutes') \
              .groupBy(window(col('time_coinapi'), '1 minutes'), col('symbol_id'), 'taker_side') \
              .agg(sum(col('size')), count(col('uuid'))) \
              .select(col('sum(size)').alias('size'), col('symbol_id'), col('window').alias('time_frame'), col('count(uuid)').alias('count'), col('taker_side'))

# Add a column consisting of 4 elements based on the split symbol_id columns' first 4 elements
trades_symbol_split = trades_aggregated.withColumn('symbol_parts', F_slice(split(col('symbol_id'), '_'), 1, 4))

# Unnest the columns and drop the nested symbol_parts column
# Then turn the columns as a whole into 1 JSON formatted string column called value
trades_clean = trades_symbol_split\
            .withColumn("broker", col('symbol_parts')[0])\
            .withColumn("symbol_type", col('symbol_parts')[1])\
            .withColumn("asset", col('symbol_parts')[2])\
            .withColumn("asset_quote", col('symbol_parts')[3])\
            .drop('symbol_parts')\
            .select(to_json(struct('*')).alias('value'))

# Every minute, write the new batch to the trades_aggregated topic
query = trades_clean.writeStream \
              .trigger(processingTime='1 minutes')\
              .format("kafka") \
              .option("kafka.bootstrap.servers", "kafka1:9093") \
              .option("checkpointLocation", "/home/jovyan/checkpoint")\
              .option("topic", "trades_aggregated") \
              .outputMode("complete") \
              .start()

query.awaitTermination()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
spark.stop()